<a href="https://colab.research.google.com/github/hjfreyer/render/blob/master/Render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install wurlitzer


In [0]:
import os
import tensorflow as tf
import collections
import pprint


In [0]:
"""Library for implict surface functions."""


def pt(x, y, z):
    return np.array([x, y, z], dtype=np.float32)

def direction(x, y, z):
    return tf.math.l2_normalize(tf.constant([x, y, z], dtype=tf.float32))

class Sphere:
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius

    def distance(self, pts):
        distance_from_center = pts - tf.expand_dims(self.center, 0)
        return tf.norm(distance_from_center, axis=1) - self.radius

class Inverse:
    def __init__(self, geometry):
        self.geometry = geometry

    def distance(self, pts):
        return -self.geometry.distance(pts)

class Plane:
    def __init__(self, point, normal):
        self.point = point
        self.normal = normal

    def distance(self, pts):
        to_ref_point = pts - tf.expand_dims(self.point, axis=0)
        along_normal = tf.reduce_sum(
            to_ref_point * tf.expand_dims(self.normal, axis=0),
            axis=1)
        return along_normal

class Intersection:
    def __init__(self, surfaces):
        self.surfaces = surfaces

    def distance(self, pts):
        stacked = tf.stack([s.distance(pts) for s in self.surfaces], axis=1)
        return tf.reduce_max(stacked, axis=1)

class Box:
    def __init__(self, p1, p2):
        small = tf.minimum(p1, p2)
        big = tf.maximum(p1, p2)

        self.planes = [Plane(big, direction(0, 0, 1)),
                       Plane(small, direction(0, 0, -1)),
                       Plane(big, direction(1, 0, 0)),
                       Plane(small, direction(-1, 0, 0)),
                       Plane(big, direction(0, 1, 0)),
                       Plane(small, direction(0, -1, 0))]

    def distance(self, pts):
        vec = tf.stack([p.distance(pts) for p in self.planes], axis=-1)
        return tf.reduce_max(vec, axis=-1)


In [0]:
# Import libraries for simulation
import tensorflow as tf
import collections
import numpy as np

# Imports for visualization
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


def pt(x, y, z):
    return np.array([x, y, z], dtype=np.float32)


def direction(x, y, z):
    return tf.math.l2_normalize(tf.constant([x, y, z], dtype=np.float32))


#camera = np.array([0, 0, 0], dtype=np.float32)
WIDTH = 1000 * 1000
HEIGHT = 1000 * 1000
CONVERGED = 0.001
SCENE_DIAMETER = 100
focal_length = 1.0

# Camera starts at the origin pointing down the x axis in the positive
# direction.
y_points = tf.range(-1,  1,  2.0 / WIDTH, dtype=tf.float32)
z_points = tf.range(1, -1, -2.0 / HEIGHT, dtype=tf.float32)
y_coords, z_coords = tf.meshgrid(y_points, z_points)
x_coords = tf.fill(y_coords.shape, tf.constant(focal_length, tf.float32))

pixels3d = tf.stack([x_coords, y_coords, z_coords], axis=2)
pixels3d = tf.reshape(pixels3d, (WIDTH * HEIGHT, 3))
ray_dirs = tf.math.l2_normalize(pixels3d, axis=1)
ray_coords = tf.zeros_like(
    ray_dirs) + tf.expand_dims(tf.constant([-1, 0, 1.3], dtype=tf.float32), axis=0)


Ray = collections.namedtuple('Ray', 'root direction')
Surface = collections.namedtuple('Surface', 'geometry color')
Emitter = collections.namedtuple('Emitter', 'source color')
Scene = collections.namedtuple('Scene', 'surfaces emitters')

rays = Ray(root=ray_coords, direction=ray_dirs)


def ray_gather_nd(ray, *args, **kwargs):
    root = tf.gather_nd(ray.root, *args, **kwargs)
    dir = tf.gather_nd(ray.direction, *args, **kwargs)
    return Ray(root, dir)


SCENE = Scene(
    surfaces=[
        Surface(geometry=Sphere(pt(0, 0, 1.3), 0.2),
                color=pt(1, 0, 1)),
        Surface(geometry=Sphere(pt(0.25, 0.75, 1.2), 0.1),
                color=pt(1, 1, 0)),
        # Surface(geometry=Intersection([Plane(pt(10, 0, 0), direction(-1, -1, 0)),
        #                                     Plane(pt(10, 0, 0), direction(0, 0, 1))]),
        #distance=plane(pt(0, 0, -0.1), direction(0, 0, 1)),
        # color=pt(0.7, 0.7, 1)),

        # Surface(geometry=Box(pt(4, -2, 0), pt(3, 1, 1)),
        #       #distance=plane(pt(0, 0, -0.1), direction(0, 0, 1)),
        #       color=pt(0, 1, 1)),
        Surface(geometry=Inverse(Box(pt(-20, -20, 0), pt(20, 20, 10))),
                color=pt(1, 1, 1)),
        Surface(geometry=Box(pt(-0.5, -1, 1), pt(0.5, 1, 1.1)),
                color=pt(1, 1, 1)),
    ],
    emitters=[
        Emitter(source=pt(0, 5, 5), color=pt(10, 10, 10)),
        Emitter(source=pt(-2, -2, 2), color=pt(0.25, 0, 0)),
    ])


def white_balance(colors):
    """Takes (N, 3), does white balance"""
    max_per_channel = tf.math.reduce_max(colors, axis=0, keepdims=True)
    colors = colors / max_per_channel

    # In case of really small values of max_per_channel causing problems.
    colors = tf.minimum(colors, 1)
    return colors


def gamma_correct(colors):
    # Formula from https://mitchellkember.com/blog/post/ray-tracer/
    linear_regime = 12.82 * colors
    exp_regime = 1.055 * colors**(1 / 2.4) - 0.055
    return tf.where(color <= 0.0031308, linear_regime, exp_regime)


def get_closest_surface_distance(rays, lengths):
    ray_ends = rays.root + rays.direction * tf.expand_dims(lengths, 1)
    surface_dists = tf.stack([s.geometry.distance(ray_ends)
                              for s in SCENE.surfaces], axis=1)
    return tf.math.reduce_min(surface_dists, axis=1), tf.argmin(surface_dists, axis=1)


def propagate_rays(geometry, rays):
    def cond(unconverged_idxes, lengths):
        return 0 < tf.size(unconverged_idxes)

    def body(unconverged_idxes, lengths):
        unconverged_rays = ray_gather_nd(rays, unconverged_idxes)
        unconverged_lengths = tf.gather_nd(lengths, unconverged_idxes)

        ray_ends = unconverged_rays.root + unconverged_rays.direction * \
            tf.expand_dims(unconverged_lengths, 1)

        surface_dist = geometry.distance(ray_ends)

        new_unconverged = tf.logical_not(tf.logical_or(
            surface_dist < CONVERGED, SCENE_DIAMETER < unconverged_lengths))
        new_unconverged_idxes = tf.boolean_mask(unconverged_idxes, new_unconverged)

        length_delta = surface_dist * tf.cast(new_unconverged, tf.float32)

        unconverged_count = tf.size(unconverged_idxes)
        with tf.control_dependencies([tf.print("Propagating: ", unconverged_count)]):
            # converged = tf.math.logical_or(
            #     converged,
            #     tf.scatter_nd(unconverged_idxes,  new_converged, converged.shape))
            lengths += tf.scatter_nd(unconverged_idxes,
                                     length_delta, lengths.shape)

        return new_unconverged_idxes, lengths

    unconverged_idxes = tf.where(tf.equal(
        tf.zeros(rays.root.shape[:-1]), 0))

    _, lengths = tf.while_loop(cond, body, (unconverged_idxes,
                                            tf.zeros(
                                                rays.root.shape[:-1], dtype=tf.float32),
                                            ))
    return lengths

def batch_propagate_rays(geometry, rays):
    roots, dirs = rays
    batches1 = tf.split(roots, 2, axis=0)
    batches2 = tf.split(dirs, 2, axis=0)
    batch_lengths = [propagate_rays(geometry, Ray(b1,b2)) for b1, b2 in zip(batches1, batches2)]
    return tf.concat(batch_lengths, axis=0)

def get_clearance(surfaces, points):
    def cond(too_close, points):
        return tf.math.reduce_any(too_close)

    def body(too_close, points):
        too_close_idxes = tf.where(too_close)
        
        with tf.control_dependencies([tf.print("Getting clearance: ", tf.size(too_close_idxes))]):
            too_close_points = tf.gather_nd(points, too_close_idxes)

        surface_dists = tf.stack([s.geometry.distance(too_close_points)
                                  for s in SCENE.surfaces], axis=1)
        min_surface_dists = tf.math.reduce_min(surface_dists, axis=1)

        gradients, = tf.gradients(min_surface_dists, too_close_points)

        new_too_close = min_surface_dists < CONVERGED
        points_delta = gradients * CONVERGED * tf.expand_dims(tf.cast(new_too_close, tf.float32), axis=-1)

        points += tf.scatter_nd(too_close_idxes, points_delta, points.shape)
        too_close = tf.math.logical_and(
            too_close,
            tf.scatter_nd(too_close_idxes, new_too_close, too_close.shape))

        return too_close, points
    _, points = tf.while_loop(cond, body, (
        tf.fill(points.shape[:1], True),
        points
    ))
    return points


with tf.Session() as sess:
    emitters_sources = tf.stack([e.source for e in SCENE.emitters], axis=0)
    emitters_colors = tf.stack([e.color for e in SCENE.emitters], axis=0)

    rays = rays._replace(root=get_clearance(SCENE.surfaces, rays.root))
    lengths = tf.stack([batch_propagate_rays(s.geometry, rays)
                        for s in SCENE.surfaces], axis=-1)
    lengths = tf.math.reduce_min(lengths, axis=-1)

    reflection_points = rays.root + rays.direction * tf.expand_dims(lengths, 1)
    reflection_points = get_clearance(SCENE.surfaces, reflection_points)

    reflection_points_to_emitters_vector = (tf.expand_dims(emitters_sources, axis=0) -
                                            tf.expand_dims(reflection_points, axis=1))
    reflection_points_to_emitters_dir = tf.math.l2_normalize(
        reflection_points_to_emitters_vector, axis=2)

    reflection_points = tf.broadcast_to(tf.expand_dims(
        reflection_points, 1), tf.shape(reflection_points_to_emitters_dir))

    reflected_rays = Ray(root=reflection_points,
                         direction=reflection_points_to_emitters_dir)
    reflected_lengths = tf.stack(
        [batch_propagate_rays(s.geometry, reflected_rays) for s in SCENE.surfaces], axis=-1)
    reflected_lengths = tf.math.reduce_min(reflected_lengths, axis=-1)
    reflected_lengths = tf.reshape(
        reflected_lengths, (-1, len(SCENE.emitters)))

    distance_to_emitters = tf.norm(
        reflection_points_to_emitters_vector, axis=2)
    unobstructed = distance_to_emitters < reflected_lengths

    ray_ends = rays.root + rays.direction * tf.expand_dims(lengths, 1)
    surface_dists = tf.stack([s.geometry.distance(ray_ends)
                              for s in SCENE.surfaces], axis=1)
    closest_surface_dist = tf.math.reduce_min(surface_dists, axis=1)
    closest_surface_arg = tf.argmin(surface_dists, axis=1)

    normals, = tf.gradients(closest_surface_dist, ray_ends)
    normals = tf.math.l2_normalize(normals, axis=1)

    emitters_dot_products = tf.reduce_sum(
        reflection_points_to_emitters_dir * tf.expand_dims(normals, axis=1),
        axis=2)
    emitters_dot_products = tf.maximum(emitters_dot_products, 0)
    emitters_dot_products *= tf.cast(unobstructed, tf.float32)
    emitters_dot_products /= distance_to_emitters**2

    emitters_colors_per_point = tf.expand_dims(
        emitters_colors, axis=0) * tf.expand_dims(emitters_dot_products, axis=2)
    surface_colors = np.stack([s.color for s in SCENE.surfaces])
    surface_color = tf.gather(surface_colors, closest_surface_arg)
    reflected_color_per_emitter = emitters_colors_per_point * \
        tf.expand_dims(surface_color, axis=1)
    color = tf.reduce_sum(reflected_color_per_emitter, axis=1)

    color = white_balance(color)
    color = gamma_correct(color)
    color = tf.reshape(color, (WIDTH, HEIGHT, 3))
    color = tf.cast(256 * color, dtype=tf.uint8)
    
    
    #color = tf.reshape(tf.cast(closest_surface_arg, tf.float32), (WIDTH, HEIGHT))

    
    
    
    #tf.global_variables_initializer().run()
    print('Variables initialized')
    from wurlitzer import sys_pipes

    with sys_pipes():
      print('Graph built')
      #color_tpu = tf.contrib.tpu.rewrite(lambda: color, [])
      #print('tpuified')
      #sess.run(tf.contrib.tpu.initialize_system())
      %time c = sess.run(color)

    try:
        Image.fromarray(c).save('render.png')
    except Exception as e:
        print(e)
    plt.imshow(c)
    plt.show()


ValueError: ignored